In [ ]:
import tensorflow as tf
print (tf.__version__)

In [ ]:
import pandas as pd
import json
import matplotlib
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os
from PIL import ImageFile
from tqdm import tqdm
import h5py
import keras
import keras.backend as K
from keras.models import Model
from keras import optimizers
from sklearn.manifold import TSNE
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras import backend as K
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, \
    GlobalMaxPool2D, Concatenate, GlobalMaxPooling2D, GlobalAveragePooling2D, Lambda
from keras.applications.inception_v3 import InceptionV3
import tensorflow as tf
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.backend.tensorflow_backend import set_session
from collections import defaultdict
import time
from PIL import Image
from functools import partial
import functools
import cv2 as cv
%pylab inline
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.callbacks import ReduceLROnPlateau
from keras.utils import multi_gpu_model

from PIL import Image

from DataGenerator import *
from metrics import *

In [ ]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()

In [ ]:
imgLst = os.listdir('../wish/train/')
nbTrain = len(imgLst)
print (nbTrain)

In [ ]:
df = pd.DataFrame()
df = pd.read_csv('labels.csv')

In [ ]:
# list of tags, labels and booleans 
tags = list(df['Unnamed: 0'].values)
lbls = list(df['labels'].values)

In [ ]:
# we iterate in tags list and we keep only the index where kp == 1
# tags = [tag for ind, tag in enumerate(tags)
#         if kp[ind] == 1]
# lbls = [lbl for ind, lbl in enumerate(lbls)
#         if kp[ind] == 1]

In [ ]:
print (len(tags), len(lbls))

In [ ]:
import json
from pprint import pprint

with open('../wish/train.json') as f:
    data = json.load(f)

In [ ]:
# delete elem which is not in tags
# def tags_to_pop(tagsLst):
#     for ind, tag in enumerate(tagsLst):
#         if tag not in tags:
#             del tagsLst[ind]
#     return (tagsLst)

In [ ]:
print ("le dataset est de {} images".format(nbTrain))
img_to_tags = {int(dic['imageId']):list(map(int, dic['labelId']))
               for ind, dic in enumerate(data['annotations'])
                if ind < nbTrain}

In [ ]:
tag_to_imgs = {}

In [ ]:
for i in tags:
    keyLst = []
    for key, value in img_to_tags.items():
        if i in value:
            keyLst.append(key)
    tag_to_imgs[i] = keyLst

In [ ]:
# imgPath list of path per image 
imgsPath = ['../wish/train/{}.jpg'.format(key)
            for key, value in img_to_tags.items()
            if len(value) > 0]
# tagsImg list[list] of tags  
tagsImg = [value for key, value in img_to_tags.items()
           if len(value) > 0]

In [ ]:
print ('number of images: {}\nnumber of tags list: {}'
       .format(len(imgsPath),len(tagsImg)))

In [ ]:
# errImg file which contains the list of images where we can get the shape
df3 = pd.read_csv('errImg.csv')
err = df3['error'].values
pos = df3['position'].values

In [ ]:
print ('number of conflicting file {}'.format(len(err)))

In [ ]:
nb = 30
tagsImg = [x for ind, x in enumerate(tagsImg)
           if imgsPath[ind] not in err]
imgsPath = [x for x in imgsPath
            if x not in err]

In [ ]:
#pour voir l'id maximum 
ma = 0
for img in imgsPath:
    nb = int(img.replace('../wish/train/', '').replace('.jpg', ''))
    if nb > ma:
        ma = nb

In [ ]:
print ('the maximum image id {}'.format(ma))

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

mlb = MultiLabelBinarizer()
cat_tag = mlb.fit_transform(tagsImg)

In [ ]:
for i in range(0, 3):
    print (tagsImg[i], cat_tag[i])

In [ ]:
(X_train, X_test, y_train, y_test) = train_test_split(imgsPath,
                                                      cat_tag,
                                                      test_size=0.1,
                                                      random_state=42)

In [ ]:
# EarlyStopping and ReduceLROnPlateau or LearningRateScheduler 
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from keras.applications.resnet50 import ResNet50
from keras.applications.mobilenet import MobileNet
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions

from keras.applications.inception_v3 import InceptionV3
from keras.layers import Input

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=[224, 224, 3])

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)

predictions = Dense(len(mlb.classes_), activation='sigmoid')(x)

In [ ]:
batch_size = 128

print ('number of classes: {}'.format(len(mlb.classes_)))

from keras.metrics import top_k_categorical_accuracy

with tf.device("/cpu:0"):
    model = Model(inputs=base_model.input, outputs=predictions)
print('number of layers : ' + str(len(model.layers)))

for layer in model.layers:
    layer.trainable = True
    if isinstance(layer, keras.layers.BatchNormalization):
        layer.momentum = 0.8
    
for layer in model.layers[:-20]:
    layer.trainable = False
    
model.compile(optimizer=keras.optimizers.adamax(lr=1e-2),
              loss = 'binary_crossentropy',
              metrics = [fmeasure, recall, precision])

index = 0
while (True):
    if not os.path.exists('logs/' + str(index)):
        os.makedirs('logs/' + str(index))
        break ;
    index += 1
    
logsName = './logs/wish' + str(index) # replace index by the name you want
tensorboard = keras.callbacks.TensorBoard(log_dir=logsName, 
                                          histogram_freq=0, 
                                          batch_size=batch_size, 
                                          write_graph=True, 
                                          write_grads=True, 
                                          write_images=True, 
                                          embeddings_freq=0,
                                          embeddings_layer_names=None, 
                                          embeddings_metadata=None)

- https://medium.com/@ilango100/batchnorm-fine-tune-your-booster-bef9f9493e22

In [ ]:
params = {
            'dim': (224,224,3),
            'batch_size': 32,
            'n_classes': len(mlb.classes_),
            'n_channels': 3,
            'shuffle': True
         }
training_generator = DataGenerator(X_train, y_train, **params)
validation_generator = DataGenerator(X_test, y_test, **params)

In [ ]:
epochs = 4
history = model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=3,
                    epochs=epochs,
                    callbacks=[tensorboard])

In [ ]:

# historique pour la loss function
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
model.summary()